In [1]:
from abc import ABC, abstractmethod  # Import abstract base class tools

# Define an abstract base class to represent a generic Runnable component
class Runnable(ABC):

    # Abstract method that all subclasses must implement
    # This method will define how the runnable is "invoked" or executed
    @abstractmethod
    def invoke(input_data):
        pass  # No implementation here – forces subclasses to implement this method

In [6]:
import random

# NakliLLM is a dummy LLM class that inherits from Runnable, so it can be invoked like other LangChain components
class NakliLLM(Runnable):

    # Constructor gets called when an instance is created
    def __init__(self):
        print('LLM created')  # Print a message when the model is instantiated

    # This method satisfies the Runnable interface — it's how LangChain will invoke this class
    def invoke(self, prompt):
        # A fixed list of mock responses (just like a fake LLM)
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        # Randomly choose a response and return it in dictionary format
        return {'response': random.choice(response_list)}

    # An alternate method (non-standard) to simulate traditional LLM behavior
    def predict(self, prompt):
        # Same list of fake responses
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        # Randomly return one
        return {'response': random.choice(response_list)}

In [2]:
# NakliPromptTemplate is a dummy prompt template class that also inherits from Runnable
class NakliPromptTemplate(Runnable):

    # Constructor to initialize the prompt template and its input variables
    def __init__(self, template, input_variables):
        self.template = template                      # The prompt pattern (e.g., "Write a {length} poem about {topic}")
        self.input_variables = input_variables        # List of expected variable names in the template

    # Standard LangChain method used when chaining components together
    def invoke(self, input_dict):
        # Takes a dictionary of input values and fills in the template
        return self.template.format(**input_dict)

    # A custom helper method that does the same thing — not required by LangChain
    def format(self, input_dict):
        # Returns the formatted prompt string
        return self.template.format(**input_dict)

In [3]:
# NakliStrOutputParser is a dummy output parser that extracts a string response from the model's output.
# It inherits from Runnable, making it chainable in LangChain-like pipelines.
class NakliStrOutputParser(Runnable):

    # Constructor – nothing to initialize in this simple case
    def __init__(self):
        pass

    # The core method used in chains – processes the input and returns the final output
    def invoke(self, input_data):
        # Expects input_data to be a dictionary with a 'response' key (as returned by NakliLLM)
        return input_data['response']

In [4]:
# RunnableConnector is a custom class to chain multiple Runnables together, like a mini pipeline builder.
# It inherits from Runnable, so it fits into the LangChain-style pipeline architecture.

class RunnableConnector(Runnable):

    # Takes a list of Runnables (e.g., prompt template → model → output parser)
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    # The invoke method processes input_data through each runnable in sequence
    def invoke(self, input_data):

        # Passes the input_data through each runnable in the list
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)  # output of one becomes input for the next

        # Final output after going through all Runnables
        return input_data

In [7]:
# Create a prompt template for poem generation
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

# Instantiate the fake LLM (simulated response generator)
llm = NakliLLM()

# Output parser that extracts just the 'response' field from the LLM output
parser = NakliStrOutputParser()

# Chain together: prompt → LLM → parser
chain = RunnableConnector([template, llm, parser])

# Run the chain with input parameters
chain.invoke({'length': 'long', 'topic': 'india'})

# --- Second example below ---

# First template: ask LLM to write a joke about the given topic
template1 = NakliPromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

# Second template: ask LLM to explain the joke using the generated response
template2 = NakliPromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

# LLM and parser again
llm = NakliLLM()
parser = NakliStrOutputParser()

# Chain 1: generate a joke → gets LLM response (dict with 'response')
chain1 = RunnableConnector([template1, llm])

# Chain 2: takes the joke, explains it → goes through LLM + parser
chain2 = RunnableConnector([template2, llm, parser])

# Final chain: runs chain1, then passes its output to chain2
final_chain = RunnableConnector([chain1, chain2])

# Run final_chain with topic "cricket"
final_chain.invoke({'topic': 'cricket'})

LLM created
LLM created


'AI stands for Artificial Intelligence'